# Extended Kalman Filter

Use uncertain information (sensor data) and knowledge of how the system behaves (educated guess) to understand the state of a system which is continuously changing)

eg, combine
- an intermittent and imprecise GPS signal
- prediction of what will _probably_ happen when you enter a certain input, such as 'forward'
- noisey but high frequency sensor data from an IMU

To get a very accurate estimate of the current position and velocity of the system

KF assumes that the variables are random and Gaussian distributed with a mean value $\mu$ and variance/uncertainty $\sigma^2$

However, KF relies on the fact that the variables involved are related in some way - eg. position and velocity

Following a guide from https://www.bzarg.com/p/how-a-kalman-filter-works-in-pictures/ Equation numbers correspond with this guide

In [1]:
import numpy as np

## The State Estimate

$$ x_k = \begin{bmatrix} position \\ velocity \end{bmatrix} $$

where $x_k$ is the best estimate of the current state and velocity

In [2]:
x = np.array([0, 0])

## The Covariance Matrix

$$ P = \begin{bmatrix} \Sigma_{pp} & \Sigma_{pv} \\ \Sigma_{vp} & \Sigma_{vv} \end{bmatrix}$$

where $ \Sigma_{ij} $ is the degree of variance between the _i'th_ and _j'th_ state variable.

In [3]:
P = np.array([[0, 0],
              [0, 0]])

## The Prediction Matrix, Control Matrix and Control Vector

Indicates how we move from one state to the next

$$ p_k = p_{k-1} + \Delta t v_{k-1} + \frac{1}{2} a \Delta t^2 $$
$$ v_k = v_{k-1} + a \Delta t$$

Thus,

$$ \begin{split} \hat{x}_k &= \begin{bmatrix} 1 & \Delta t \\ 0 & 1 \end{bmatrix}\hat{x}_{k-1} + \begin{bmatrix} \frac{\Delta t^2}{2} \\ \Delta t \end{bmatrix} a \\ &= F_k\ \hat{x}_{k-1} + B_k\ u_k \end{split} $$

where $F_k$ is the prediction matrix, $B_k$ is the control matrix, and $u_k$ is the control vector

$B$ and $u$ represented tracked external influences

In other words, the new best estimate is a prediction made from the previous best estimate, plus a correction for known external influences

In [5]:
T = 0.1
F = np.array([[1, T],
              [0, 1]])
B = np.array([[T**2 / 2],
               [T]])

## Factoring in Uncertainty

Eg. uncertainty from modelling, noise, etc

Update the prediction matrix as follows,

$$ P_k = F_k\ P_{k-1}\ F_k^T + Q_k$$

In other words, the new uncertainty is predicted from the old uncertainty, with some additional uncertainty from the environment

## The Sensor Uncertainty Matrix

$$ \mu_{expected} = H_k\ \hat{x}_k $$
$$ \Sigma_{expected} = H_k\ P_k\ H_k^T $$

where $H_k$ is a matrix which models the sensors

In [24]:
def update_state_predict(F, x_prev, B, u):             # equation 7
    x_cur = (F @ x_prev) + (B @ u)
    return x_cur

def update_prediction_matrix_predict(F, P_prev, Q):    # equation 7
    P_cur = F @ P_prev @ F.T + Q
    return P_cur

def update_state_update(x_cur, K, z, H):             # equation 18
    x_best = x_cur + K @ (z - H @ x_cur)
    return x_best

def update_prediction_matrix_update(P, K, H):    # equation 18
    P_best = P_cur - K @ H @ P_cur
    return P_best

def update_Kalman_gain(P, H, R):            # equation 19
    K_new = P @ H.T @ np.invert(H @ P @ H.T + R)
    return K_new


# x, P update the thing goes along
# measure u


- x, P update as the thing goes along

- F, B is modelled

- u is measured

- z is the sensor measurement???

K???
H???
R???
Q???

In [ ]:
class KalmanFilter():
    """
        x       the state estimate
        P       the covariance matrix
        F       the prediction matrix
        B       the control matrix
        K       the Kalman gain matrix
    """
    self.x = np.array([0, 0])
    self.P = np.array([[0, 0],
                       [0, 0]])

    def __init__(self, F, B):
        self.F = F
        self.B = B
    
    def predict(self, u, Q):
        """ get from x[k-1] to x[k] by using knowledge of how the system behaves """
        self.x = (self.F @ self.x) + (self.B @ u)
        self.P = (self.F @ self.P @ self.F.T) + Q

    def update(self, H, R, z):
        """ get from x[k] to the best possible estimate of the system by using sensor data """
        self.K = (self.P @ H.T) @ np.invert(H @ P @ H.T + R)
        self.x = self.x + (K @ (z - H @ self.x))
        self.P = self.P - (K @ H @ self.P)
    
    def get_predictions(self):
        return self.x